# **ETL - ATLÁNTICO**

In [1]:
# [Configuración] Importación de librerías
import pandas as pd
import numpy as np
import unicodedata
import re
import os

In [2]:
# [Configuración] Lectura de los archivos CSV
ruta = "../DATA/DNG" 
archivos = [f for f in os.listdir(ruta) if f.endswith(".csv")]
dataframes = {}
for archivo in archivos:
    path = os.path.join(ruta, archivo)
    df = pd.read_csv(path, encoding='latin-1', low_memory=False)
    df.columns = df.columns.str.lower()
    dataframes[archivo] = df

print(f"✅ {len(dataframes)} archivos cargados desde {ruta}")

✅ 7 archivos cargados desde ../DATA/DNG


In [3]:
# [Función] Limpiar nombres de columnas
def limpiar_texto(texto):
    if not isinstance(texto, str):
        return texto
    texto = texto.replace("ã±", "ñ").replace("Ã±", "ñ").replace("ï¿½", "ñ")
    texto = unicodedata.normalize("NFKC", texto)
    texto = re.sub(r'\s+', '_', texto.strip().lower())
    texto = texto.replace("_", "")
    return texto

# Limpiar nombres de columnas de todos los DataFrames
for nombre, df in dataframes.items():
    df.columns = [limpiar_texto(col) for col in df.columns]
    dataframes[nombre] = df


columnas_por_archivo = {
    nombre: set(df.columns)
    for nombre, df in dataframes.items()
}
# Columnas comunes a todos los archivos
comunes = set.intersection(*columnas_por_archivo.values())

# Columnas únicas por archivo
exclusivas = {
    nombre: columnas - comunes
    for nombre, columnas in columnas_por_archivo.items()
}
print("✅ Columnas comunes a todos los archivos:")
print(sorted(comunes))

print("\n❌ Columnas exclusivas por archivo:")
for nombre, cols in exclusivas.items():
    print(f"📄 {nombre}: {sorted(cols)}")

✅ Columnas comunes a todos los archivos:
['ajuste', 'area', 'año', 'barver', 'cbmte', 'cenpobla', 'cerdef', 'codase', 'coddptoo', 'codeve', 'codmuno', 'codpaiso', 'codpre', 'codsub', 'confin', 'dirres', 'edad', 'fecaju', 'fecarcxl', 'feccon', 'fecdef', 'fechanto', 'fechos', 'fecnot', 'fmfuerza', 'fmgrado', 'fmunidad', 'gpcarcela', 'gpdesmovi', 'gpdesplaz', 'gpdiscapa', 'gpgestan', 'gpindigen', 'gpmadcom', 'gpmigrant', 'gpotros', 'gppobicbf', 'gppsiquia', 'gpvicvio', 'inisin', 'localidad', 'ndepnotif', 'ndepproce', 'ndepresi', 'nitupgd', 'nmunnotif', 'nmunproce', 'nmunresi', 'nomdilf', 'nomeve', 'nomupgd', 'numide', 'ocupacion', 'pachos', 'peretn', 'priape', 'prinom', 'segape', 'segnom', 'semana', 'sexo', 'teldilf', 'telefono', 'tipcas', 'tipide', 'tipss', 'unimed', 'vereda']

❌ Columnas exclusivas por archivo:
📄 raw_2017.csv: ['coddptor', 'codmunr', 'evehistorico']
📄 raw_2018.csv: ['coddptor', 'codmunr', 'evehistorico']
📄 raw_2019.csv: ['coddptor', 'codmunr', 'evehistorico']
📄 raw_2020

In [4]:
# Descartar variables poco informativas
delete = [
    'ajuste', 'barver', 'cbmte', 'cenpobla', 'cerdef', 'codase', 'coddptor', 'codpaiso','codmunr', 'codpre', 'codsub', 'dirres', 'evehistorico','fecaju', 'fecarcxl',
    'fmfuerza', 'fmgrado', 'fmunidad','fecdef','fechanto', 'numide', 'priape', 'prinom','segape','segnom', 'cod_dep', 'cod_mun', 'codeve',
    'gpcarcela', 'gpdesmovi', 'gpdesplaz', 'gpdiscapa', 'gpgestan', 'gpindigen', 'gpmadcom', 'gpmigrant', 'gpotros', 'gppobicbf', 'gppsiquia', 'gpvicvio', 
    'localidad', 'ndepnotif', 'ndepresi', 'nitupgd', 'nmunnotif', 'nmunresi', 'nomdilf', 'nomupgd', 'ocupacion', 'teldilf', 'telefono', 'tipide', 'vereda',
]

for nombre, data in dataframes.items():
    cols_a_borrar = [col for col in delete if col in data.columns]
    if cols_a_borrar:
        dataframes[nombre] = data.drop(columns=cols_a_borrar)
        print(f"🗑️ En '{nombre}' se eliminaron: {cols_a_borrar}")
    else:
        print(f"✅ En '{nombre}' no había columnas para eliminar.")

🗑️ En 'raw_2017.csv' se eliminaron: ['ajuste', 'barver', 'cbmte', 'cenpobla', 'cerdef', 'codase', 'coddptor', 'codpaiso', 'codmunr', 'codpre', 'codsub', 'dirres', 'evehistorico', 'fecaju', 'fecarcxl', 'fmfuerza', 'fmgrado', 'fmunidad', 'fecdef', 'fechanto', 'numide', 'priape', 'prinom', 'segape', 'segnom', 'codeve', 'gpcarcela', 'gpdesmovi', 'gpdesplaz', 'gpdiscapa', 'gpgestan', 'gpindigen', 'gpmadcom', 'gpmigrant', 'gpotros', 'gppobicbf', 'gppsiquia', 'gpvicvio', 'localidad', 'ndepnotif', 'ndepresi', 'nitupgd', 'nmunnotif', 'nmunresi', 'nomdilf', 'nomupgd', 'ocupacion', 'teldilf', 'telefono', 'tipide', 'vereda']
🗑️ En 'raw_2018.csv' se eliminaron: ['ajuste', 'barver', 'cbmte', 'cenpobla', 'cerdef', 'codase', 'coddptor', 'codpaiso', 'codmunr', 'codpre', 'codsub', 'dirres', 'evehistorico', 'fecaju', 'fecarcxl', 'fmfuerza', 'fmgrado', 'fmunidad', 'fecdef', 'fechanto', 'numide', 'priape', 'prinom', 'segape', 'segnom', 'codeve', 'gpcarcela', 'gpdesmovi', 'gpdesplaz', 'gpdiscapa', 'gpgestan

## Unificación

In [5]:
# Concatenar archivos en un solo DataFrame
data = pd.concat(dataframes.values(), ignore_index=True)
renombres = {
    "coddptoo": "cod_dep",
    "codmuno": "cod_mun",
    "confin":"desenlace",
    "ndepproce": "departamento",
    "nmunproce": "municipio",
    "fecnot": "fecha_notific",
    "feccon": "fecha_consulta",
    "fechos": "fecha_hospital",
    "inisin": "fecha_ini_sint",
    "nomeve": "evento",
    "pachos": "hospitalizado",
    "peretn": 'etnia',
    "tipcas": "tipo_caso",
    "tipss": "regimen_salud",

}
data.rename(columns=renombres, inplace=True)
data = data[sorted(data.columns)]
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33170 entries, 0 to 33169
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   area            33170 non-null  int64 
 1   año             33170 non-null  int64 
 2   cod_dep         33170 non-null  int64 
 3   cod_mun         33170 non-null  int64 
 4   departamento    33170 non-null  object
 5   desenlace       33170 non-null  int64 
 6   edad            33170 non-null  int64 
 7   etnia           33170 non-null  int64 
 8   evento          33170 non-null  object
 9   fecha_consulta  33170 non-null  object
 10  fecha_hospital  33170 non-null  object
 11  fecha_ini_sint  33170 non-null  object
 12  fecha_notific   33170 non-null  object
 13  hospitalizado   33170 non-null  int64 
 14  municipio       33170 non-null  object
 15  regimen_salud   33170 non-null  object
 16  semana          33170 non-null  int64 
 17  sexo            33170 non-null  object
 18  tipo_c

_________

## Agregación

In [6]:
# [Variable] Evento
data.loc[:, 'evento'] = data['evento'].str.strip()
data['evento'] = np.where(data['evento'] == 'DENGUE', 'Clasico', 
                       np.where(data['evento'] == 'DENGUE GRAVE', 'Grave', 'Otros'))
data['evento'].unique()

array(['Clasico', 'Grave'], dtype=object)

In [7]:
# [Variable] Edad en años
data = data[data['edad'] != 0]
data["edad_años"] = np.where(data["unimed"] == 1, data["edad"],   # ya está en años
                    np.where(data["unimed"] == 2, data["edad"] / 12,  # meses → años
                    np.where(data["unimed"] == 3, data["edad"] / 365, # días → años
                    np.where(data["unimed"] == 4, data["edad"] / (365*24), # horas
                    np.where(data["unimed"] == 5, data["edad"] / (365*24*60), # minutos
                    np.where(data["unimed"] == 0, np.nan, 
                     np.nan))))))  # por si hay otros valores
data["edad_años"] = data["edad_años"].round().fillna(-1).astype(int)

In [8]:
# [Variable] Ciclo vital a partir de edad en años
def ciclo_vital(edad):
    if edad < 0:
        return "Desconocido"
    elif edad < 1:
        return "Menor de 1 año"
    elif edad < 5:
        return "Primera infancia"
    elif edad < 10:
        return "Infancia"
    elif edad < 20:
        return "Adolescencia"
    elif edad < 60:
        return "Adulto"
    elif edad < 85:
        return "Adulto mayor"
    else:
        return "Ancianidad"
    
data["ciclo_vital"] = data["edad_años"].apply(ciclo_vital)
data["ciclo_vital"].unique()

array(['Ancianidad', 'Adolescencia', 'Adulto mayor', 'Infancia', 'Adulto',
       'Primera infancia', 'Menor de 1 año'], dtype=object)

In [9]:
# [Mapeo] Variables categóricas
mapeo = {
    'area' : {1: 'Cabecera municipal' , 2:'Centro poblado', 3:'Rural disperso'}, 
    'hospitalizado': {1: 'Si', 2: 'No'},
    'desenlace': {1: 'Vivo', 2: 'Fallecido'},
    'etnia': {1: 'Indigena', 2: 'Rom', 3: 'Raizal', 4: 'Palenquero', 5: 'Negro', 6:'Otro'},
    'regimen_salud': {'P': 'Excepción', 'E': 'Especial', 'C': 'Contributivo', 'S': 'Subsidiado', 'N': 'No asegurado', 'I': 'Indeterminado'},
    'sexo': {'M': 'Masculino', 'F': 'Femenino'},
    'tipo_caso': {2: 'Probable', 3: 'Conf. por laboratorio', 5: 'Conf. por nexo'}
}
data = data.replace(mapeo)

In [10]:
# [Mapeo] Grupos étnicos
mapeo_etnico = {
    'Indigena': 'Indígena',
    'Negro': 'Afrocolombiano',
    'Raizal': 'Afrocolombiano',
    'Palenquero': 'Afrocolombiano',
    'Rom': 'Rom',
    'Otro': 'Otro'
}
data['grupo_etnico'] = data['etnia'].map(mapeo_etnico)

In [11]:
# [Variable] Departamento
data['departamento'] = data["departamento"].str.upper().str.strip()
data['departamento'] = data['departamento'].str.replace('_', ' ', regex=False)
data['departamento'] = data['departamento'].replace({'ATLANTICO': 'ATLÁNTICO'})
data = data[data['departamento'] == "ATLÁNTICO"]
data['departamento'].unique()

array(['ATLÁNTICO'], dtype=object)

In [12]:
# [Variable] Departamento
data['municipio'] = data["municipio"].str.upper().str.strip()
data['municipio'] = data['municipio'].str.replace('_', ' ', regex=False)

muni = {
    'REPELON': 'REPELÓN',
    'USIACURI': 'USIACURÍ',
    'REPELON': 'REPELÓN', 
    'SANTO TOMAS': 'SANTO TOMÁS', 
    'TUBARA': 'TUBARÁ',
    '* ATLANTICO. MUNICIPIO DESCONO': 'DESCONOCIDO',
    'MANATI': 'MANATÍ',
    'POLO NUEVO': 'POLONUEVO',
    'PIOJO': 'PIOJÓ',
    'SANTA LUCIA': 'SANTA LUCÍA'
}

data['municipio'] = data['municipio'].replace(muni)
data = data[data['municipio'] != "DESCONOCIDO"]
data['municipio'].unique()

array(['BARRANQUILLA', 'SOLEDAD', 'REPELÓN', 'JUAN DE ACOSTA', 'GALAPA',
       'USIACURÍ', 'BARANOA', 'POLONUEVO', 'SABANALARGA', 'SABANAGRANDE',
       'MALAMBO', 'LURUACO', 'CAMPO DE LA CRUZ', 'SANTO TOMÁS', 'TUBARÁ',
       'PONEDERA', 'PUERTO COLOMBIA', 'MANATÍ', 'CANDELARIA',
       'PALMAR DE VARELA', 'SUAN', 'PIOJÓ', 'SANTA LUCÍA'], dtype=object)

In [13]:
def obtener_subregion(mun):
    if mun in ['BARRANQUILLA', 'GALAPA', 'MALAMBO', 'SOLEDAD', 'PUERTO COLOMBIA']:
        return 'Metropolitana'
    elif mun in ['PALMAR DE VARELA', 'PONEDERA', 'SANTO TOMÁS', 'SABANAGRANDE']:
        return 'Oriental'
    elif mun in ['CAMPO DE LA CRUZ', 'CANDELARIA', 'SANTA LUCÍA', 'SUAN', 'REPELÓN', 'MANATÍ']:
        return 'Sur'
    elif mun in ['BARANOA', 'POLONUEVO', 'LURUACO', 'USIACURÍ', 'SABANALARGA']:
        return 'Centro'
    elif mun in ['JUAN DE ACOSTA', 'PIOJÓ', 'TUBARÁ']:
        return 'Costera'
    else:
        return 'Desconocida'


# Aplicamos la función
data['subregion'] = data['municipio'].apply(obtener_subregion)

In [14]:
# [Variable] Periodo epidemiológico (4 semanas)
def asignar_periodo(semana):
    """
    Calcula el número de periodo según la semana epidemiológica.
    Cada periodo abarca 4 semanas. 
    La semana 53 se incluye en el último periodo.
    """
    if pd.isna(semana):
        return np.nan
    semana = int(semana)
    periodo = int(np.ceil(semana / 4))
    if semana == 53:  # Semana extra del año
        periodo = int(np.ceil(52 / 4))  # igual al último periodo
    return periodo

# Aplicar al dataframe
data["periodo"] = data["semana"].apply(asignar_periodo)

In [15]:
duplicados = data[data.duplicated()]
print(f"🔍 Filas duplicadas: {len(duplicados)}")

🔍 Filas duplicadas: 91


In [16]:
data.columns

Index(['area', 'año', 'cod_dep', 'cod_mun', 'departamento', 'desenlace',
       'edad', 'etnia', 'evento', 'fecha_consulta', 'fecha_hospital',
       'fecha_ini_sint', 'fecha_notific', 'hospitalizado', 'municipio',
       'regimen_salud', 'semana', 'sexo', 'tipo_caso', 'unimed', 'edad_años',
       'ciclo_vital', 'grupo_etnico', 'subregion', 'periodo'],
      dtype='object')

In [17]:
# Descartar variables originales ya transformadas
data = data.drop(columns=['edad', 'unimed', 'edad_años', 'etnia', 'departamento'])

In [18]:
# [Data] Estructuración por temática

    # Variables demográficas
demograficas = [
    "sexo",                    # M/F
    "edad_años",               # Edad numérica
    "ciclo_vital",             # Etapa de vida
    "grupo_etnico",            # Grupo étnico declarado
    "regimen_salud"            # Régimen (contributivo, subsidiado, etc.)
]

    # Variables del evento epidemiológico
evento = [
    "evento",                  # Clásico, grave
    "tipo_caso",               # Confirmado, probable, descartado
    "hospitalizado",            # Sí/No
    "desenlace"                # Recuperado, fallecido, etc.
]
'fecha_consulta', 'fecha_hospital', 'fecha_ini_sint', 'fecha_notific',
    # Variables de temporalidad
temporalidad = [
    "año",                      # Año del evento
    "semana",                   # Semana epidemiológica
    "periodo",                  # Periodo (cada 4 semanas)
    "fecha_ini_sint",           # Inicio de síntomas
    "fecha_consulta",           # Fecha de consulta
    "fecha_hospital",           # Fecha de hospitalización
    "fecha_notific",            # Fecha de notificación
]

    # Variables espaciales
espacial = [
    "municipio",                # Municipio de ocurrencia
    "subregion",                # Subregion del Atlántico
    "area",                     # Urbana o rural
]

orden_columnas = demograficas + evento + temporalidad + espacial
cols_existentes = [col for col in orden_columnas if col in data.columns]
data = data[cols_existentes + [col for col in data.columns if col not in cols_existentes]]

In [19]:
# DataFrame final
data.shape

(32533, 20)

In [20]:
df_atl = data[data['municipio'] != "BARRANQUILLA"]
df_atl.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12888 entries, 7 to 30018
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   sexo            12888 non-null  object
 1   ciclo_vital     12888 non-null  object
 2   grupo_etnico    12888 non-null  object
 3   regimen_salud   12888 non-null  object
 4   evento          12888 non-null  object
 5   tipo_caso       12888 non-null  object
 6   hospitalizado   12888 non-null  object
 7   desenlace       12888 non-null  object
 8   año             12888 non-null  int64 
 9   semana          12888 non-null  int64 
 10  periodo         12888 non-null  int64 
 11  fecha_ini_sint  12888 non-null  object
 12  fecha_consulta  12888 non-null  object
 13  fecha_hospital  12888 non-null  object
 14  fecha_notific   12888 non-null  object
 15  municipio       12888 non-null  object
 16  subregion       12888 non-null  object
 17  area            12888 non-null  object
 18  cod_dep    

In [21]:
df_atl.to_csv(r"C:\Users\Hp\DENGUE\DATA\Data_Atlantico.csv", index=False, encoding='utf-8')